In [ ]:
import polars as pl

: 

In [ ]:
UNSPSC_DATA_PATH = "../data/UNSPSC English v260801.xlsx"

In [ ]:
df = pl.read_excel(UNSPSC_DATA_PATH, read_options={"header_row": 12})

In [ ]:
df = df.drop("Synonym", "Acronym", "Version", "Key")

In [ ]:
df = df.rename(
    {
        "Segment": "segment_code",
        "Segment Title": "segment_title",
        "Segment Definition": "segment_definition",
        "Family": "family_code",
        "Family Title": "family_title",
        "Family Definition": "family_definition",
        "Class": "class_code",
        "Class Title": "class_title",
        "Class Definition": "class_definition",
        "Commodity": "commodity_code",
        "Commodity Title": "commodity_title",
        "Segment Definition": "segment_definition",
        "Family Definition": "family_definition",
        "Class Definition": "class_definition",
        "Commodity Definition": "commodity_definition",
    }
)

In [ ]:
df.filter(pl.col('segment_code').cast(str).str.starts_with('24'))

In [ ]:
import json
dicts = (
    df
    .filter(
        (
            pl.col("commodity_code") == 40161513) 
            | (pl.col("commodity_code") == 40101802) 
            | (pl.col("commodity_code") == 40141607) 
            | (pl.col("commodity_code") == 20121301)
        )
    .to_dicts()
)
json_lines = "\n".join([json.dumps(d) for d in dicts])
with open("unspsc_data.json", "w") as f:
    f.write(json_lines)


In [ ]:
descriptions = [
    "FILTER,CARTRIDGE: WATER SEPARATOR,FUEL,3.866IN ID,4.575IN OD,12.323IN LG,10U,22 TO 24 GPM,98.7%,GLASS/PAPER ELEMENT,1-1/4IN-12 UNS-2B SPIN-ON,SAE J1985,SIZE: 4.736IN OD SEAM,W/ GASKET",
    "EXCHANGER,HEAT: 2IN,TITANIUM,GASKETED,150PSIG,SINGLE",
    "VALVE,BALL: 1IN,NPT,FULL PORT BORE,316 STAINLESS STEEL BODY,STAINLESS STEEL STEM,17-4PH STAINLESS STEEL BALL,17-4PH STAINLESS STEEL STEM,HANDLE OPERATED,D2STAINLESS STEELF-GIS,PRESSURE RATING: 3000PSI,SOFTGOODS: PTFE TAILPIECE GASKET,SOFTGOODS: HOMWPOLYMER ACETAL PLASTIC SEAL,SERIES 310",
    "PLUG,PIPE: TEST,2IN,FLANGE,2-9/16IN,STEEL,GALVANIZED,WING NUT,W/ MOLDED RUBBER GASKET"
]

In [ ]:
import json
with open("Unspsc_data.json", "r") as f:
    data = json.load(f)


In [ ]:
print(list(df.sample(10).select(pl.col("commodity_definition")).to_dict().values())[0][0])

In [ ]:
def create_search_text(_row) -> str:
    fields = [
        _row["segment_title"],
        _row["segment_definition"],
        _row["family_title"],
        _row["family_definition"],
        _row["class_title"],
        _row["class_definition"],
        _row["commodity_title"],
        _row["commodity_definition"],
    ]
    return re.sub(
        r"\s+", " ", " ".join(str(field) for field in fields if field).strip()
    ).lower()


df = df.with_columns(
    pl.struct(
        "segment_title",
        "segment_definition",
        "family_title",
        "family_definition",
        "class_title",
        "class_definition",
        "commodity_title",
        "commodity_definition",
    )
    .map_elements(lambda x: create_search_text(x), return_dtype=pl.String)
    .alias("combined_text")
)

In [ ]:
df.filter(pl.col("commodity_code") == 20121301).to_dicts()